# Combined Model
Combines the encoder and classifier to create the full model.

In [1]:
import tensorflow as tf
print(f"GPU is {'not ' if len(tf.config.list_physical_devices('GPU')) == 0 else ''}available")

2024-04-29 14:20:42.464428: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 14:20:43.014551: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is available


2024-04-29 14:20:43.822267: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 14:20:43.847218: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 14:20:43.847264: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import keras

# Data Processing

In [3]:
import pandas as pd

The previous 3 notebooks should already been ran, so we should be able to read this file without any problems.

In [4]:
raw_df = pd.read_csv("../data/dataset.csv")
raw_df

,label,hash,dim-0000,dim-0001,dim-0002,dim-0003,dim-0004,dim-0005,dim-0006,dim-0007,...,dim-9990,dim-9991,dim-9992,dim-9993,dim-9994,dim-9995,dim-9996,dim-9997,dim-9998,dim-9999
0,TRICKBOT,f16631469eb35406ef4049d30c763cadda571b25bbdb45...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,DARKKOMET,d31a7102cbc54447c251ba62760eb484fd0c9fbb8ea54f...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,INSTALLBRAIN,a5ba68828e571de66675befdf4fbaf26dd226e25c2c703...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UNKNOWN,c40861e5ebd3c30de810f33c0959aaf5683586fe819998...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,WALDEK,26714e389dbb964ddd764ee4f1bceaf56b18adc8734668...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,TRICKBOT,9506421d996290f70689559ee0c09cc074c948fff49547...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1269,NECONYD,0eee965f286f057a3175797590795bbf99fda65dc8d845...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1270,TRICKBOT,0c6aa0ae05d5fa8bf5a8ea95310be73ee60e55a0ce6864...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1271,UNKNOWN,08b4e7389242e3b8c37215a3b972f69193a9a12d5130bf...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


For training the model, we don't need the `hash`.

In [5]:
df = raw_df.drop(columns=["hash"])
df

,label,dim-0000,dim-0001,dim-0002,dim-0003,dim-0004,dim-0005,dim-0006,dim-0007,dim-0008,...,dim-9990,dim-9991,dim-9992,dim-9993,dim-9994,dim-9995,dim-9996,dim-9997,dim-9998,dim-9999
0,TRICKBOT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,DARKKOMET,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,INSTALLBRAIN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UNKNOWN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,WALDEK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,TRICKBOT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1269,NECONYD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1270,TRICKBOT,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1271,UNKNOWN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Load the label encoder now so that we can see the labels that it encoded.

In [6]:
import pickle

with open("../models/classifier/label-encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

In [7]:
label_encoder.classes_

array(['AGENTB', 'AGENTTESLA', 'BENIGN', 'BERBEW', 'BLACKMATTER',
       'BLADABINDI', 'BULZ', 'CERBER', 'COBALTSTRIKE', 'CRIMSONRAT',
       'DARKKOMET', 'DRIDEX', 'EMOTET', 'FARFLI', 'GAFGYT', 'GAMARUE',
       'GANDCRAB', 'HIVE', 'HUPIGON', 'ICEDID', 'INSTALLCORE',
       'JAKYLLHYDE', 'KOLABC', 'MANSABO', 'MANUSCRYPT', 'MIMDAU', 'MIRAI',
       'MOSAICREGRESSOR', 'MSILHERACLES', 'NANOCORE', 'NECONYD', 'NOON',
       'NUKESPED', 'NYMAIM', 'NYMERIA', 'PAJETBIN', 'PHORPIEX', 'PLUGX',
       'PREPSCRAM', 'RELINE', 'REMCOS', 'ROZENA', 'SAGENT', 'SALITY',
       'TNEGA', 'TOFSEE', 'TRICKBOT', 'VIRLOCK', 'VOBFUS', 'WANNACRY',
       'WAPOMI', 'ZAPCHAST'], dtype=object)

Remove any row that does not have a label within these classes.

In [8]:
df = df[df["label"].isin(label_encoder.classes_)].reset_index(drop=True)
df

,label,dim-0000,dim-0001,dim-0002,dim-0003,dim-0004,dim-0005,dim-0006,dim-0007,dim-0008,...,dim-9990,dim-9991,dim-9992,dim-9993,dim-9994,dim-9995,dim-9996,dim-9997,dim-9998,dim-9999
0,TRICKBOT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,DARKKOMET,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,HIVE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,AGENTB,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,RELINE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,FARFLI,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
664,TRICKBOT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
665,NECONYD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
666,TRICKBOT,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


Separate into `X` and `y`.

In [9]:
X = df.drop(columns=["label"])
y = df["label"]

In [10]:
print(X.shape)
print(y.shape)

(668, 10000)
(668,)


# Label Encoding and Train-Test Split

Using the loaded label encoder, encode the labels in `y`.

In [11]:
y = label_encoder.transform(y)

80% of the data will be saved for training, while 20% will be left for testing.

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(534, 10000)
(134, 10000)
(534,)
(134,)


# Model Creation

In [14]:
from keras import layers
from keras.models import Sequential

The full model is also quite simple - it is just the `Encoder` head combined with the `Classifier` tail.

In [15]:
encoder = keras.models.load_model("../models/encoder/encoder.keras")
classifier = keras.models.load_model("../models/classifier/classifier.keras")

2024-04-29 14:20:45.219531: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 14:20:45.219647: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 14:20:45.219681: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 14:20:46.041452: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 14:20:46.041523: I external/local_xla/xla/stream_executor

In [16]:
INITIAL_EPOCH = 0

def create_model():
    model = Sequential(name="MACBETh")

    model.add(keras.Input((X.shape[1],)))
    model.add(encoder)
    model.add(classifier)

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=1e-5),  # We just want to fine-tune the model, so no need high LR
        metrics=["sparse_categorical_accuracy"]
    )
    return model


model = create_model()
model.summary(expand_nested=True)

Model: "MACBETh"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Encoder (Sequential)            │ (None, 32)             │     5,288,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense (Dense)              │ (None, 512)            │     5,120,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_1 (Dense)            │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_2 (Dense)            │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_3 (Dense)            │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Classifier (Sequential)         │ (None, 52)             │        38,580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense (Dense)              │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_1 (Dense)            │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_2 (Dense)            │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_3 (Dense)            │ (None, 52)             │         3,380 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,327,444 (20.32 MB)

 Trainable params: 5,327,444 (20.32 MB)

 Non-trainable params: 0 (0.00 B)

Train the model slightly (this is just for fine-tuning).

In [17]:
model.fit(
    X_train,
    y_train,
    epochs=10
)

Epoch 1/10


I0000 00:00:1714371647.768739 1081003 service.cc:145] XLA service 0x7fe21c00fdc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714371647.768809 1081003 service.cc:153]   StreamExecutor device (0): Quadro P1000, Compute Capability 6.1
2024-04-29 14:20:47.798272: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-29 14:20:48.601147: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


16/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8609 - sparse_categorical_accuracy: 0.7971

I0000 00:00:1714371651.336505 1081003 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


17/17 ━━━━━━━━━━━━━━━━━━━━ 9s 250ms/step - loss: 0.8719 - sparse_categorical_accuracy: 0.7947
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9572 - sparse_categorical_accuracy: 0.7915
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0295 - sparse_categorical_accuracy: 0.7497
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8584 - sparse_categorical_accuracy: 0.8010
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9976 - sparse_categorical_accuracy: 0.7746
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8662 - sparse_categorical_accuracy: 0.7912
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8883 - sparse_categorical_accuracy: 0.7911
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9556 - sparse_categorical_accuracy: 0.7719
Epoch 9/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9289 - sparse_categorical_accuracy: 0.7723
Epoch 10/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8938 - sparse_categor

Check the model's performance.

In [18]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f"Testing loss:      {test_loss:5.5f}")
print(f"Testing accuracy:  {test_acc:5.5f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step - loss: 1.9198 - sparse_categorical_accuracy: 0.5533
Testing loss:      1.94545
Testing accuracy:  0.55970


In [19]:
model.save("../models/combined/combined.keras")